In [ ]:
# default_exp exec.plotting

In [ ]:
# export
from montlake.atomgeom.features import get_features,get_D_feats_feats
from montlake.atomgeom.utils import get_atoms_4
from montlake.simulations.rigidethanol import get_rigid_ethanol_data
from montlake.utils.utils import get_234_indices, get_atoms3_full, get_atoms4_full, data_stream_custom_range, get_cosines
from montlake.geometry.geometry import get_geom, get_wlpca_tangent_sel, get_rm_tangent_sel
from montlake.gradients.estimate import get_grads_pullback
from montlake.statistics.normalization import normalize_L212
from montlake.optimization.gradientgrouplasso import get_sr_lambda_parallel
from montlake.optimization.utils import get_selected_function_ids,get_selected_functions_lm2
from montlake.utils.replicates import Replicate, get_supports_brute
from montlake.plotting.manifolds import plot_manifold_2d,plot_manifold_featurespace
from montlake.plotting.flasso import plot_reg_path_ax_lambdasearch_customcolors_norm, plot_watch_custom
from megaman.embedding import SpectralEmbedding
from montlake.statistics.supportrecovery import get_min_min, get_mu_full_ind, get_kappa_s,get_gamma_max
from montlake.plotting.main import plot_experiment

import pandas as pd
import dill as pickle
import os
import sys
import numpy as np
import itertools
from itertools import permutations,combinations
from sklearn.decomposition import TruncatedSVD
import pathos
from pathos.multiprocessing import ProcessingPool as Pool

In [ ]:
# export
import numpy as np
import random
import argparse
import json
import sys
import scipy
from ..vendor.tfcompat.hparam import HParams
import os

def parse_args(args):
    parser= argparse.ArgumentParser()
    parser.add_argument("--config", help="Path to JSON config (can override with cmd line args)")
    #parser.add_argument("--raw_data", help="Path to raw data")
    #parser.add_argument("--outdir", help="Path to save outputs")
    args = parser.parse_args(args)
    return args


In [ ]:
# export

try:
    from nbdev.imports import IN_NOTEBOOK
except:
    IN_NOTEBOOK = False
    
if __name__ == "__main__" and not IN_NOTEBOOK:
    
    np.random.seed(1234)
    random.seed(1234)
    args = parse_args(sys.argv[1:])
    config = {}
    if args.config:
        with open(args.config) as f:
            config.update(json.load(f))

    config.update(vars(args))
    hparams = HParams(**config)
    if not os.path.exists(hparams.outdir):
        os.makedirs(hparams.outdir)
        
    positions = np.load(hparams.position_file)
    for key in hparams.ground_truth.keys():
        if hparams.ground_truth[key] is None:
            hparams.ground_truth[key] = np.asarray([])
        hparams.ground_truth[key] = np.asarray(hparams.ground_truth[key], dtype = int)
        
    print('plotting',hparams.ground_truth)
    #if not hparams.color_counts_all:
    #    hparams.color_counts_all = None
    
    #hparams.color_counts_all = hparams.color_counts_all or None
   # hparams.colors_id_all = hparams.colors_id_all or None
#     kwargs = dict(result_file = hparams.result_file, 
#                     positions = positions, 
#                     d = hparams.d,
#                     name = hparams.name, 
#                     ncord = hparams.ncord,
#                     embedding = hparams.embedding,
#                     ground_truth = hparams.ground_truth, 
#                     ground_truth_colors = hparams.ground_truth_colors,
#                     outdir = hparams.outdir,
#                     color_counts_all = hparams.color_counts_all,
#                    colors_id_all = hparams.colors_id_all,
#                    n_components = hparams.n_components,
#                    ptsize = hparams.ptsize,
#                    alpha = hparams.alpha)
#    plot_experiment(**{k: v for k, v in kwargs.items() if v is not None})
#    plot_experiment(**{k: v for k, v in kwargs.items() if v is not None})
    plot_experiment(result_file = hparams.result_file, 
                    positions = positions, 
                    d = hparams.d,
                    name = hparams.name, 
                    ncord = hparams.ncord,
                    embedding = hparams.embedding,
                    ground_truth = hparams.ground_truth, 
                    colors_gt = hparams.ground_truth_colors,
                    outdir = hparams.outdir,
                    color_counts_all = hparams.color_counts_all,
                   colors_id_all = hparams.colors_id_all,
                   n_components = hparams.n_components,
                   ptsize = hparams.ptsize,
                   alpha = hparams.alpha,
                   gt_reg_color = hparams.gt_reg_color,
                   sel_reg_color = hparams.sel_reg_color,
                   plot_watch_full = hparams.plot_watch_full,
                   plot_watch_results = hparams.plot_watch_results,
                    cosine_color = hparams.cosine_color,
                    selected_cosines = hparams.selected_cosines,
                    cosine_cluster = hparams.cosine_cluster
                   )
        

In [11]:
import json
with open("/Users/samsonkoelle/tunatostada/experiments/configs_plotting/rigidethanol_diagram.json") as f:
    asdf = json.load(f)

In [50]:
        print('loading data')
        import dill as pickle
        with open("/Users/samsonkoelle/thesis_data/processed_data_2/ethanol/eth_full_mfresults_mflasso",'rb') as inp:
            results = pickle.load(inp, pickle.HIGHEST_PROTOCOL)

        ground_truth =  {"atoms2": None,
                    "atoms3": None, 
                    "atoms4" : [[6,1,0,2],[1,0,2,8]]}
        
        superset = results['dictionary']['atoms4']
        positions = np.load("/Users/samsonkoelle/thesis_data/processed_data_2/ethanol/positionseth_full_mf.npy")
        natoms = positions.shape[1]
        selected_lasso = results['selected_lasso']
        #atoms4 = np.asarray(list(itertools.combinations(range(natoms), 4))) 
        superset = results['dictionary']['atoms4']#get_atoms4_full(atoms4) #needs adjustment for diagram dictionaries
        j1 = get_index_matching(ground_truth['atoms4'][0], superset) #needs adjustment for non torsion ground truths
        j2 = get_index_matching(ground_truth['atoms4'][1], superset)
        gt_ind = [j1,j2]
        names_gt = get_names(np.asarray(gt_ind))

NameError: name 'get_index_matching' is not defined

In [ ]:
            selected_lasso_gt = np.unique(np.concatenate((gt_ind, selected_lasso))) #add 234
            #detected_values = get_detected_values2d(selected_lasso_gt, results['supports_lasso'][0],nreps)
            colors_lasso_plusgt = np.vstack([colors_gt, colors_lasso])
            names_lasso_plusgt = np.concatenate((names_gt, names_lasso))
            cuz_l = np.abs(get_cosines(results['replicates_small'][0].dg_M[:,:,selected_lasso_gt]))
            cuz_l0 = np.mean(cuz_l, axis = 0)
            fig, axarr = plt.subplots(figsize=(15, 10))
            sns.heatmap(cuz_l0, yticklabels = names_lasso_plusgt, xticklabels = names_lasso_plusgt, ax = axarr, vmin = 0., vmax = 1.)
            axarr.set_xticklabels(axarr.get_xmajorticklabels(), fontsize = 30)
            axarr.set_yticklabels(axarr.get_ymajorticklabels(), fontsize = 30)
            if d == 2:
                for r in range(nreps):
                    axarr.add_patch(Rectangle((results['supports_lasso'][1][r,1], results['supports_lasso'][1][r,0]), 1, 1,facecolor = [0,1,0,0.], hatch = '/',fill= True, edgecolor='blue', lw=1))
                    axarr.add_patch(Rectangle((results['supports_lasso'][1][r,0], results['supports_lasso'][1][r,1]), 1, 1,facecolor = [0,1,0,0.], hatch = '/',fill= True, edgecolor='blue', lw=1))


In [43]:
# def supportlist_to_tensor(supports,p ):
#     d = supports.shape[1]
#     nreps = supports.shape[0]
#     support_tensor = np.zeros(np.repeat(p,d))
#     for r in range(25):
#         toadd = np.asarray(list(permutations(supports[r],d)))
#         for s in range(len(toadd)):
#             support_tensor[tuple(np.asarray(toadd[s], dtype = int))] += 1
#     return(support_tensor)

In [27]:
import numpy as np
from itertools import permutations